In [10]:
import pandas as pd
import numpy as np


df =  pd.read_csv("tyres_model.csv")
# df.head()

In [4]:
print(df.shape)

(3000, 16)


In [6]:
df[df.duplicated()]
# there are no duplicated data

,vulc,perc_nat_rubber,wiring_strength,weather,perc_imp,temperature,tread_type,tyre_season,elevation,month,tread_depth,tyre_quality,perc_exp_comp,diameter,add_layers,failure


In [8]:
# check for missing values
df.isna().any()
# we see that only diameter has nan values

vulc               False
perc_nat_rubber    False
wiring_strength    False
weather            False
perc_imp           False
temperature        False
tread_type         False
tyre_season        False
elevation          False
month              False
tread_depth        False
tyre_quality       False
perc_exp_comp      False
diameter            True
add_layers         False
failure            False
dtype: bool

In [14]:
df.isna().head()
# results shows 5 rows which contains NA values, but then only 2 have True as results
df[df.isna().any(axis=1)]
# maybe add an histogram to show the comparison between the number of nas and the numbers of actual values

,vulc,perc_nat_rubber,wiring_strength,weather,perc_imp,temperature,tread_type,tyre_season,elevation,month,tread_depth,tyre_quality,perc_exp_comp,diameter,add_layers,failure
1,20.704,36,1,0.30,0.01,-4.52,2,0,328.0,11,1,0,6.15,NaN,1,0
2,19.156,34,1,0.30,0.01,-1.08,0,0,247.0,0,1,0,6.36,NaN,1,0
5,20.042,38,0,0.04,0.01,-0.20,2,1,160.5,0,0,0,5.96,NaN,1,0
6,21.172,33,1,0.39,0.01,-2.28,0,0,470.0,3,0,0,6.47,NaN,1,0
7,16.706,32,0,0.62,0.05,-3.96,3,1,148.5,7,2,0,5.39,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,17.860,21,1,0.16,0.01,-6.48,0,1,336.5,8,0,1,5.85,NaN,1,0
2994,19.298,29,1,0.03,0.00,-1.00,2,1,167.5,8,0,0,5.82,NaN,2,0
2995,17.818,29,2,0.39,0.01,7.28,1,1,287.5,10,1,1,5.68,NaN,1,1
2996,17.076,30,1,0.22,0.00,-1.44,1,1,152.5,6,1,0,5.81,NaN,1,1


In [19]:
df_drop =  df.dropna(axis=1)
df_drop.head()
print(df_drop.shape)

(3000, 15)


## Feature selection

Now we should choose and look at every feature and how it changes, to exclude predictives variables that do not change too much

In [20]:
# now data should be fine, but check it
# we should now look at distribution and values on all predictive variables

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

y0 = df[df["failure"] == 0]
y1 = df[df["failure"] == 1]

fig, axes = plt.subplot()